In [1]:
import tqdm

def nop(it, *a, **k):
    return it

real_tqdm = tqdm.tqdm
tqdm.tqdm = nop
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import time
import glob
import pickle

import numpy as np
np.bool = np.bool_
import cv2
import torch
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import torch.nn.functional as F

from utils.inference.image_processing import crop_face, get_final_image, show_images, normalize_and_torch, normalize_and_torch_batch
from utils.inference.video_processing import read_video, get_target, get_final_video, add_audio_from_another_video, face_enhancement, crop_frames_and_get_transforms, resize_frames
from utils.inference.core import model_inference, transform_target_to_torch
from utils.inference.faceshifter_run import faceshifter_batch
from network.AEI_Net import AEI_Net
from coordinate_reg.image_infer import Handler
from insightface_func.face_detect_crop_multi import Face_detect_crop
from arcface_model.iresnet import iresnet100
from models.pix2pix_model import Pix2PixModel
from models.config_sr import TestOptions

/compuworks/anaconda3/envs/xaiseung_ghost_cuda114/lib/python3.9/site-packages/kornia/augmentation/augmentation.py:1830: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  warnings.warn(
/compuworks/anaconda3/envs/xaiseung_ghost_cuda114/lib/python3.9/site-packages/mxnet/numpy/fallback.py:143: DeprecationWarning: In accordance with NEP 32, the function mirr was removed from NumPy version 1.20.  A replacement for this function is available in the numpy_financial library: https://pypi.org/project/numpy-financial
  mirr = onp.mirr
/compuworks/anaconda3/envs/xaiseung_ghost_cuda114/lib/python3.9/site-packages/mxnet/numpy/fallback.py:160: DeprecationWarning: In accordance with NEP 32, the function npv was removed from NumPy version 1.20.  A replacement for this function is available in the numpy_financial library: https://pypi.org/project/numpy-financial
  npv = onp.npv
/compuworks/anaconda3/envs/xaise

### Load Models

In [2]:
app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id= 0, det_thresh=0.6, det_size=(640,640))

# main model for generation
G = AEI_Net(backbone='unet', num_blocks=2, c_id=512)
G.eval()
G.load_state_dict(torch.load('weights/G_unet_2blocks.pth', map_location=torch.device('cpu')))
G = G.cuda()
G = G.half()

# arcface model to get face embedding
netArc = iresnet100(fp16=False)
netArc.load_state_dict(torch.load('arcface_model/backbone.pth'))
netArc=netArc.cuda()
netArc.eval()

# model to get face landmarks
handler = Handler('./coordinate_reg/model/2d106det', 0, ctx_id=0, det_size=640)

# model to make superres of face, set use_sr=True if you want to use super resolution or use_sr=False if you don't
use_sr = True
if use_sr:
    torch.backends.cudnn.benchmark = True
    opt = TestOptions()
    #opt.which_epoch ='10_7'
    model = Pix2PixModel(opt)
    model.netG.train()

input mean and std: 127.5 127.5
find model: ./insightface_func/models/antelope/glintr100.onnx recognition
find model: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx detection
set det-size: (640, 640)
loading ./coordinate_reg/model/2d106det 0
input mean and std: 127.5 127.5
find model: ./insightface_func/models/antelope/glintr100.onnx recognition
find model: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx detection
set det-size: (640, 640)


[04:09:09] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v1.5.0. Attempting to upgrade...
[04:09:09] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[04:09:09] ../src/base.cc:79: cuDNN lib mismatch: linked-against version 8302 != compiled-against version 8101.  Set MXNET_CUDNN_LIB_CHECKING=0 to quiet this warning.


Network [LIPSPADEGenerator] was created. Total number of parameters: 72.2 million. To see the architecture, do print(network).
Load checkpoint from path:  weights/10_net_G.pth


### Set here path to source image and video for faceswap

# 특징 빼기 + 인젝션

In [3]:
import torch
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm

from utils.inference.faceshifter_run import faceshifter_batch
from utils.inference.image_processing import crop_face, normalize_and_torch, normalize_and_torch_batch
from utils.inference.video_processing import read_video, crop_frames_and_get_transforms, resize_frames
from utils.inference.core import transform_target_to_torch

image_to_image = True

set_target = False
half=True
similarity_th=0.15

crop_size = 224 # don't change this
BS = 60

# pca

In [4]:
with open("./pca_pkl/netArcPCA.pkl", "rb") as file:
    pca = pickle.load(file)


In [ ]:
from insightface.utils import face_align
import ipywidgets as widgets

set_target = False
half=True
similarity_th=0.15

crop_frames_list = None
target_batch_rs = None
start_id = 0
tmp_embed = [np.load("./examples/arc_embeds/VggFace2-crop/{}.npy".format(start_id))]
pca_array = pca.transform(tmp_embed)
with open("./pca_pkl/netArcPCAMinMax.pkl", "rb") as file:
    pca_minmax = pickle.load(file)
pca_min = pca_minmax["min"]
pca_max = pca_minmax["max"]

def load_target(path_to_target='examples/images/tgt1.jpg'):
    global crop_frames_list, target_batch_rs
    target_full = cv2.imread(path_to_target)
    full_frames = [target_full]
    target = get_target(full_frames, app, crop_size)

    target_norm = normalize_and_torch_batch(np.array(target))
    target_embeds = netArc(F.interpolate(target_norm, scale_factor=0.5, mode='bilinear', align_corners=True))

    # Get the cropped faces from original frames and transformations to get those crops
    crop_frames_list, tfm_array_list = crop_frames_and_get_transforms(full_frames,
                                                                    target_embeds,
                                                                    app,
                                                                    netArc,
                                                                    crop_size,
                                                                    set_target,
                                                                    similarity_th=similarity_th
                                                                    )
    resized_frs, present = resize_frames(crop_frames_list[0])
    resized_frs = np.array(resized_frs)

    target_batch_rs = transform_target_to_torch(resized_frs, half=True)


load_target()


def inject_drawing():
    global crop_frames_list, pca_array
    plt.figure(num=1, clear=True, figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(crop_frames_list[0][0][:,:,::-1])
    plt.title("Target Face")
    plt.subplot(1, 2, 2)

    modified_embed = pca.inverse_transform(pca_array)

    source_embed = torch.from_numpy(modified_embed).half().to("cuda")
    #source_embed[0][0] = 10.0

    Y_st = faceshifter_batch(source_embed, target_batch_rs, G)
    torch.cuda.empty_cache()

    plt.imshow(Y_st[0][:, :, ::-1])
    plt.title("After Swap")
    plt.show()

pca_sliders = []

iact_plot = widgets.interactive(
    inject_drawing
)

for i in range(40):
    pca_slider = widgets.FloatSlider(
        value=pca_array[0, i], 
        min = pca_min[i],
        max = pca_max[i],
        description=f"PCA #{i}",
        continuous_update=False,
        layout=widgets.Layout(width="300px"),
    )
    pca_slider.idx = i
    pca_slider.observe(lambda change: exec("pca_array[0, change.owner.idx]=change.new;iact_plot.update()"), names="value")
    #pca_slider.observe(lambda change: print(change.owner.idx), names="value")
    pca_sliders.append(pca_slider)
    

slider_multibox = widgets.HBox(
    children=[
    widgets.VBox(
    children = pca_sliders[:20]),
    widgets.VBox(
    children = pca_sliders[20:]),
    ]
)


display(iact_plot, slider_multibox)


# PCA 값에 따른 변화 그림

In [ ]:
fig_save_dir = "./pca_figs"
os.makedirs(fig_save_dir, exist_ok=True)


set_target = False
half=True
similarity_th=0.15
crop_frames_list = None
target_batch_rs = None

with open("./pca_pkl/netArcPCA.pkl", "rb") as file:
    pca = pickle.load(file)

with open("./pca_pkl/netArcPCAMinMax.pkl", "rb") as file:
    pca_minmax = pickle.load(file)
pca_min = pca_minmax["min"]
pca_max = pca_minmax["max"]

start_id = 0
#tmp_embed = [np.load("./examples/arc_embeds/VggFace2-crop/{}.npy".format(start_id))]
#pca_array = pca.transform(tmp_embed)

tmp_embed = [((0.3+0.7*np.random.random(pca_min.shape[0]))*(pca_max-pca_min)+pca_min)]
#tmp_embed = [(pca_min+pca_max)/2]
pca_array = pca.transform(tmp_embed)


def load_target(path_to_target='examples/images/tgt1.png'):
    global crop_frames_list, target_batch_rs
    target_full = cv2.imread(path_to_target)
    full_frames = [target_full]
    target = get_target(full_frames, app, crop_size)

    target_norm = normalize_and_torch_batch(np.array(target))
    target_embeds = netArc(F.interpolate(target_norm, scale_factor=0.5, mode='bilinear', align_corners=True))

    # Get the cropped faces from original frames and transformations to get those crops
    crop_frames_list, tfm_array_list = crop_frames_and_get_transforms(full_frames,
                                                                    target_embeds,
                                                                    app,
                                                                    netArc,
                                                                    crop_size,
                                                                    set_target,
                                                                    similarity_th=similarity_th
                                                                    )
    resized_frs, present = resize_frames(crop_frames_list[0])
    resized_frs = np.array(resized_frs)

    target_batch_rs = transform_target_to_torch(resized_frs, half=True)


load_target()

num_plotted_pca = 40
n_ticks = 5

plt.figure(num=1, clear=True, figsize=(n_ticks*2, num_plotted_pca*2))
plt.subplot(num_plotted_pca+1,n_ticks, 1)
plt.imshow(crop_frames_list[0][0][:,:,::-1])
plt.title("Target Face")
plt.axis("off")
plt.subplot(num_plotted_pca+1,n_ticks,2)

modified_embed = pca.inverse_transform(pca_array)

source_embed = torch.from_numpy(modified_embed).half().to("cuda")
#source_embed[0][0] = 10.0

Y_st = faceshifter_batch(source_embed, target_batch_rs, G)
torch.cuda.empty_cache()

plt.imshow(Y_st[0][:, :, ::-1])
plt.title("Swapped (Random)")
#plt.axis("off")
for side in ["top", "right", "bottom", "left"]: plt.gca().spines[side].set_visible(False)
plt.xticks([])
plt.yticks([])



for pci_i in range(num_plotted_pca):
    interval = (pca_max[pci_i] - pca_min[pci_i]) / (n_ticks-1)
    for c in range(n_ticks):
        plt.subplot(num_plotted_pca+1,n_ticks, (pci_i+1)*n_ticks+c+1)
        new_pca_array = pca_array.copy()
        new_pca_array[0, pci_i] = pca_min[pci_i] + (interval * c)
        modified_embed = pca.inverse_transform(new_pca_array)

        source_embed = torch.from_numpy(modified_embed).half().to("cuda")
        #source_embed[0][0] = 10.0

        Y_st = faceshifter_batch(source_embed, target_batch_rs, G)
        torch.cuda.empty_cache()

        plt.imshow(Y_st[0][:, :, ::-1])
        #plt.title("Swapped Face ")
        #plt.axis("off")
        for side in ["top", "right", "bottom", "left"]: plt.gca().spines[side].set_visible(False)
        plt.xticks([])
        plt.yticks([])
        plt.xlabel(f"{new_pca_array[0, pci_i]:.2f}")

        if c == 0:
            plt.ylabel("PCA #{}".format(pci_i))

plt.tight_layout()
save_path = f"{fig_save_dir}/PCA_for_arc_embed_to{num_plotted_pca}.png"
plt.savefig(save_path)
plt.close()
print(f"saved at {save_path}")

In [ ]:
# for comparison
tgt1_path = "./examples/images/tgt1.png"
tgt2_path = "./examples/images/elon_musk.jpg"
tgt3_path = "./examples/images/tgt2.png"


set_target = False
half=True
similarity_th=0.15
crop_size = 224 # don't change this
BS = 60


crop_frames_list = None
target_batch_rs = None
with open("./pca_pkl/netArcPCA.pkl", "rb") as file:
    pca = pickle.load(file)

with open("./pca_pkl/netArcPCAMinMax.pkl", "rb") as file:
    pca_minmax = pickle.load(file)
pca_min = pca_minmax["min"]
pca_max = pca_minmax["max"]

start_id = 0
#tmp_embed = [np.load("./embeds/{}.npy".format(start_id))]
#pca_array = pca.transform(tmp_embed)

tmp_embed = [((0.4+0.6*np.random.random(pca_min.shape[0]))*(pca_max-pca_min)+pca_min)]
#tmp_embed = [(pca_min+pca_max)/2]
pca_array = pca.transform(tmp_embed)


def load_target(path_to_target='examples/images/tgt1.png'):
    global crop_frames_list, target_batch_rs
    target_full = cv2.imread(path_to_target)
    full_frames = [target_full]
    target = get_target(full_frames, app, crop_size)

    target_norm = normalize_and_torch_batch(np.array(target))
    target_embeds = netArc(F.interpolate(target_norm, scale_factor=0.5, mode='bilinear', align_corners=True))

    # Get the cropped faces from original frames and transformations to get those crops
    crop_frames_list, tfm_array_list = crop_frames_and_get_transforms(full_frames,
                                                                    target_embeds,
                                                                    app,
                                                                    netArc,
                                                                    crop_size,
                                                                    set_target,
                                                                    similarity_th=similarity_th
                                                                    )
    resized_frs, present = resize_frames(crop_frames_list[0])
    resized_frs = np.array(resized_frs)

    target_batch_rs = transform_target_to_torch(resized_frs, half=True)


load_target(path_to_target="examples/images/tgt1.png")

num_plotted_pca = 20
n_ticks = 7




plt.figure(num=1, clear=True, figsize=(n_ticks*2, (num_plotted_pca*3+1)*2))

def draw_top(subplot_tuple, img):
    plt.subplot(*subplot_tuple)
    plt.imshow(img)
    plt.title("Target Face")
    plt.axis("off")


def draw_swap(subplot_tuple, pca_embed, is_top=False):
    global target_batch_rs, G
    plt.subplot(*subplot_tuple)

    modified_embed = pca.inverse_transform(pca_embed)
    source_embed = torch.from_numpy(modified_embed).half().to("cuda")

    Y_st = faceshifter_batch(source_embed, target_batch_rs, G)
    torch.cuda.empty_cache()
    plt.imshow(Y_st[0][:, :, ::-1])
    
    for side in ["top", "right", "bottom", "left"]: plt.gca().spines[side].set_visible(False)
    plt.xticks([])
    plt.yticks([])
    if is_top:
        plt.title("Swapped (Random)")


load_target(path_to_target=tgt1_path)
draw_top((num_plotted_pca*3+1, n_ticks, 1), crop_frames_list[0][0][:,:,::-1])
draw_swap((num_plotted_pca*3+1, n_ticks, 2), pca_array, is_top=True)

load_target(path_to_target=tgt2_path)
draw_top((num_plotted_pca*3+1, n_ticks, 3), crop_frames_list[0][0][:,:,::-1])
draw_swap((num_plotted_pca*3+1, n_ticks, 4), pca_array, is_top=True)

load_target(path_to_target=tgt3_path)
draw_top((num_plotted_pca*3+1, n_ticks, 5), crop_frames_list[0][0][:,:,::-1])
draw_swap((num_plotted_pca*3+1, n_ticks, 6), pca_array, is_top=True)

for pci_i in range(num_plotted_pca):
    interval = (pca_max[pci_i] - pca_min[pci_i]) / (n_ticks-1)
    for c in range(n_ticks):
        new_pca_array = pca_array.copy()
        new_pca_array[0, pci_i] = pca_min[pci_i] + (interval * c)
        
        load_target(tgt1_path)
        draw_swap((num_plotted_pca*3+1, n_ticks, (pci_i*3+1)*n_ticks+c+1), new_pca_array)
        plt.xlabel(f"{new_pca_array[0, pci_i]:.2f}")
        if c == 0:
            plt.ylabel("PCA #{}".format(pci_i))
        load_target(tgt2_path)
        draw_swap((num_plotted_pca*3+1, n_ticks, (pci_i*3+2)*n_ticks+c+1), new_pca_array)
        plt.xlabel(f"{new_pca_array[0, pci_i]:.2f}")
        if c == 0:
            plt.ylabel("PCA #{}".format(pci_i))
        load_target(tgt3_path)
        draw_swap((num_plotted_pca*3+1, n_ticks, (pci_i*3+3)*n_ticks+c+1), new_pca_array)
        plt.xlabel(f"{new_pca_array[0, pci_i]:.2f}")
        if c == 0:
            plt.ylabel("PCA #{}".format(pci_i))


plt.tight_layout()
save_path = f"{fig_save_dir}/PCA_for_arc_embed_Triple_to{num_plotted_pca}.png"
plt.savefig(save_path)
plt.close()
print(f"saved at {save_path}")